In [1]:
from fastai.vision.all import *
from pathlib import Path
import os, sys
sys.path.append(os.path.abspath('..'))# Add parent directory to path

from refactor_experiments import getDatasets, getData, random_seed
from refactor_losses import MixedLoss
from refactor_metrics import MetricsCallback, getDatasetMetrics
from fastai.callback.all import CSVLogger, SaveModelCallback
from config import *

sys.path.append('../../text-segmentation')

# from models.text_segmentation import TextSegament, XceptionTextSegment

import torch
# torch.cuda.set_device(0)
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Apple Silicon GPU)")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using MPS (Apple Silicon GPU)


In [2]:
EXPERIMENT_PATH = Path(EXPERIMENTS_PATH) / 'model'
MODELS_PATH = EXPERIMENT_PATH / "models"
os.makedirs(MODELS_PATH, exist_ok=True)



In [3]:
allData = getData()
print(len(allData))
datasets = getDatasets(allData)  # This returns a list of DataLoaders (one per fold)

450


In [4]:
props = {'bs': 4, 'val_bs': 2, 'num_workers': 0}
modelDict = {'resnet34': models.resnet34,}
propsOverride = {
    # 'xception': {'bs': 2},
    # 'segament': {'bs': 2}
}

In [11]:
for name, model_func in modelDict.items():
    for index, dls in enumerate(datasets):
        PATH = EXPERIMENT_PATH / name / str(index)
        PATH.mkdir(parents=True, exist_ok=True)
        if not (PATH / 'final_model.pth').exists():
            overrides = propsOverride.get(name, {})
            random_seed(42)
            print("dls.metrics:", getattr(dls, 'metrics', None))
            # dls is already a DataLoaders object
            learn = unet_learner(
                dls, model_func, 
                model_dir='models', 
                loss_func=MixedLoss(0, 1), 
                metrics=Dice(),
                cbs=[MetricsCallback(), CSVLogger()], 
                # cbs=[MetricsCallback()], 
                path=PATH
            )
            learn.model.to(device)
            random_seed(42)
            print("learn.metrics:", getattr(learn, 'metrics', None))
            print("learn.recorder.metric_names:", getattr(learn.recorder, 'metric_names', None))
            learn.fit_one_cycle(10, 1e-4)
            learn.save('model')
            learn.export('final_model.pkl')
            
for name, model_func in modelDict.items():
    for index, dls in enumerate(getDatasets(allData, crop=False, cutInHalf=False)):
        PATH = EXPERIMENT_PATH / name / str(index)
        if not (PATH / 'final_predictions.csv').exists():
            learn = load_learner(PATH, fname='final_model.pkl')
            random_seed(42)
            m = getDatasetMetrics(dls, learn)
            m.save(PATH / 'final_predictions.csv')

dls.metrics: None
learn.metrics: [<fastai.learner.AvgMetric object at 0x303479a90>]
learn.recorder.metric_names: None


TypeError: Exception occured in `CSVLogger` when calling event `before_fit`:
	sequence item 3: expected str instance, NoneType found

In [ ]:
# import segmentation_models_pytorch as smp

# props = {'bs': 4, 'val_bs': 2, 'num_workers': 0}
# models = ['resnet50', 'dpn68', 'vgg16', 'densenet169', 'efficientnet-b4']
# propsOverride = {}
# archs = [smp.Unet, smp.Linknet, smp.FPN, smp.PSPNet, smp.PAN]
# for arch in archs:
#     for model in models:
#         if model in ['vgg16', 'densenet169'] and smp.PAN == arch: #not supported
#             continue
#         for index, dataset in enumerate(getDatasets(allData, padding = 16)):
#             PATH = EXPERIMENT_PATH / (model + ' ' + arch.__name__) / str(index) 
#             if not (PATH / 'final model.pkl').exists():
#                 overrides = {} if model not in propsOverride else propsOverride[model]
#                 random_seed(42)
#                 data = dataset.databunch(**{**props, **overrides}).normalize(imagenet_stats)
#                 random_seed(42)
#                 learn = Learner(data, arch(model, encoder_weights='imagenet'), callback_fns=[MetricsCallback, CSVLogger, partial(SaveModelCallback, monitor="ignore global f1 score %")], model_dir='models', loss_func=MixedLoss(0, 1), path=PATH)
#                 random_seed(42)
#                 #freeze encoder, still not implemented in smp
#                 if hasattr(learn.model, 'reset'): learn.model.reset()
#                 for l in flatten_model(learn.model.encoder):
#                     requires_grad(l, isinstance(l, bn_types))
#                 learn.create_opt(defaults.lr)
#                 random_seed(42)
#                 learn.fit_one_cycle(10, 1e-4)
#                 learn.save('model')
#                 learn.export(file='final model.pkl')
#         for index, dataset in enumerate(getDatasets(allData, crop=False, cutInHalf=False, padding = 16)):
#             PATH = EXPERIMENT_PATH / (model + ' ' + arch.__name__) / str(index) 
#             if not (PATH / 'final predictions.csv').exists():
#                 learn = load_learner(PATH, 'final model.pkl')
#                 random_seed(42)
#                 m = getDatasetMetrics(dataset, learn)
#                 m.save(PATH / 'final predictions.csv')                

In [ ]:
# PATH = EXPERIMENT_PATH / 'xception'  
# for index, dataset in enumerate(getDatasets(allData, crop=False, cutInHalf = False)):
#     learn = load_learner(PATH / str(index)  , 'final model.pkl')

#     for idx in range(len(dataset.valid.x.items)):
#         img = dataset.valid.x.items[idx]
#         TENSOR_PATH = PATH / 'predictions' / img.parent.name / img.name.replace(path.suffix, '.pt')
#         (PATH / 'predictions' / img.parent.name).mkdir(parents=True, exist_ok=True) 
#         if not (TENSOR_PATH).exists():
#             pred = learn.predict(dataset.valid.x.get(idx, False))[2]
#             torch.save(pred, TENSOR_PATH)   